In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import random

In [2]:
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data Preprocessing

In [3]:
raw = ["I feel hungry.	나는 배가 고프다.",
       "Pytorch is very easy.	파이토치는 매우 쉽다.",
       "Pytorch is a framework for deep learning.	파이토치는 딥러닝을 위한 프레임워크이다.",
       "Pytorch is very clear to use.	파이토치는 사용하기 매우 직관적이다."]

In [4]:
SOS_token = 0 # Start Of Sentence
EOS_token = 1 # End Of Sentence

In [5]:
# filter out the long sentence from source and target data
def filter_pair(pair, source_max_length, target_max_length):
    return len(pair[0].split(" ")) < source_max_length and len(pair[1].split(" ")) < target_max_length

In [6]:
def preprocess(corpus, source_max_length, target_max_length):
    print("reading corpus...")
    pairs = []
    for line in corpus:
        pairs.append([s for s in line.strip().lower().split("\t")])
    print("Read {} sentence pairs".format(len(pairs)))
    
    pairs = [pair for pair in pairs if filter_pair(pair, source_max_length, target_max_length)]
    print("Trimmed to {} sentence pairs".format(len(pairs)))
    
    source_vocab = Vocab()
    target_vocab = Vocab()
    
    print("Counting words...")
    for pair in pairs:
        source_vocab.add_vocab(pair[0])
        target_vocab.add_vocab(pair[1])
    print("source vocab size = ", source_vocab.n_vocab)
    print("target vocab size = ", target_vocab.n_vocab)
    
    return pairs, source_vocab, target_vocab

In [7]:
class Vocab:
    def __init__(self):
        self.vocab2index = {"<SOS>":SOS_token, "<EOS>": EOS_token}
        self.index2vocab = {SOS_token:"<SOS>" , EOS_token :"<EOS>"}
        self.vocab_count = {}                 # 단어의 갯수
        self.n_vocab = len(self.vocab2index)  # 단어의 길이
        
    def add_vocab(self, sentense):
        for word in sentense.split(" "):
            # 만약 문장의 시작이나 끝인지 아닌지 검사
            if word not in self.vocab2index:          # 만약 새로운 단어가 들어 왔다면,
                self.vocab2index[word] = self.n_vocab  # 단어의 길이를 index로 사용
                self.vocab_count[word] = 1             # 단어가 몇 개 있는지 count
                self.index2vocab[self.n_vocab] = word  # 해당 index에 단어를 넣는다.
                self.n_vocab += 1                      # 단어의 길이 + 1
            
            else:                                      # 만약 vocab2index에 있는 단어라면
                self.vocab_count[word] += 1            # 해당 단어 count + 1

## Neural Net Setting

In [8]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder,self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
    def forward(self, x, hidden):
        x = self.embedding(x).view(1,1, -1)
        x, hidden = self.gru(x, hidden)
        return x, hidden
    
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, hidden):
        x = self.embedding(x).view(1,1,-1)
        x,hidden = self.gru(x, hidden)
        x = self.softmax(self.out(x[0]))
        return x, hidden

## Training

In [9]:
def tensorize(vocab, sentence):
    indexes = [vocab.vocab2index[word] for word in sentence.split(" ")]
    indexes.append(vocab.vocab2index["<EOS>"])
    return torch.Tensor(indexes).long().to(device).view(-1,1)

def train(pairs, source_vocab, target_vocab, encoder, decoder, n_iter, print_every = 1000, learning_rate = 0.01):
    loss_total = 0
    
    encoder_optimizer = optim.SGD(encoder.parameters(), lr = learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr = learning_rate)
    
    training_batch = [random.choice(pairs) for _ in range(n_iter)]
    training_source = [tensorize(source_vocab ,pair[0]) for pair in training_batch]
    training_target = [tensorize(target_vocab ,pair[1]) for pair in training_batch]
    
    criterion = nn.NLLLoss()
    
    for i in range(1, n_iter + 1):
        source_tensor = training_source[i - 1]
        target_tensor = training_target[i - 1]
        
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        source_length = source_tensor.size(0)
        target_length = target_tensor.size(0)
        
        loss = 0
        for enc_input in range(source_length):
            _, encoder_hidden = encoder(source_tensor[enc_input], encoder_hidden)
        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
        decoder_hidden = encoder_hidden
        
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
        loss.backward()
        
        encoder_optimizer.step()
        decoder_optimizer.step()
        
        loss_iter = loss.item() / target_length
        loss_total += loss_iter
        
        if i %  print_every == 0:
            loss_avg = loss_total / print_every
            loss_total = 0
            print("[{} - {}%] loss = {:05.4f}".format(i, i / n_iter * 100, loss_avg))

In [10]:
# def evaluate func
def evaluate(pairs, source_vocab, target_vocab, encoder, decoder, target_max_length):
    for pair in pairs:
        print(">", pair[0])  #print source vocab
        print("=", pair[1])  #print target vocab
        source_tensor = tensorize(source_vocab, pair[0])  # To Tensor
        source_length = source_tensor.size()[0]           # len of source_length
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)
        
        # execute Encoder every source vocab
        for ei in range(source_length):
            _, encoder_hidden = encoder(source_tensor[ei], encoder_hidden)

        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)           
        decoder_hidden = encoder_hidden
        decoded_words = []
        
        # execute Decoder every target vocab
        for di in range(target_max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            _, top_index = decoder_output.data.topk(1)    # ouput data에서 top 1 의 data만
            if top_index.item() == EOS_token:             # 문장이 끝날 때까지 반복
                decoded_words.append("<EOS>")
                break
            else:
                decoded_words.append(target_vocab.index2vocab[top_index.item()])

            decoder_input = top_index.squeeze().detach()

        predict_words = decoded_words
        predict_sentence = " ".join(predict_words)
        print("<", predict_sentence)
        print("")

In [11]:
SOURCE_MAX_LENGTH = 10
TARGET_MAX_LENGTH = 12
load_pairs, load_source_vocab, load_target_vocab = preprocess(raw, SOURCE_MAX_LENGTH, TARGET_MAX_LENGTH)
print(random.choice(load_pairs))

enc_hidden_size = 16
dec_hidden_size = enc_hidden_size
enc = Encoder(load_source_vocab.n_vocab, enc_hidden_size).to(device)
dec = Decoder(dec_hidden_size, load_target_vocab.n_vocab).to(device)

train(load_pairs,load_source_vocab, load_target_vocab, enc, dec, 5000, print_every = 1000)
evaluate(load_pairs, load_source_vocab, load_target_vocab, enc,dec,TARGET_MAX_LENGTH)


reading corpus...
Read 4 sentence pairs
Trimmed to 4 sentence pairs
Counting words...
source vocab size =  17
target vocab size =  13
['i feel hungry.', '나는 배가 고프다.']
[1000 - 20.0%] loss = 0.7422
[2000 - 40.0%] loss = 0.1070
[3000 - 60.0%] loss = 0.0339
[4000 - 80.0%] loss = 0.0181
[5000 - 100.0%] loss = 0.0124
> i feel hungry.
= 나는 배가 고프다.
< 나는 배가 고프다. <EOS>

> pytorch is very easy.
= 파이토치는 매우 쉽다.
< 파이토치는 매우 쉽다. <EOS>

> pytorch is a framework for deep learning.
= 파이토치는 딥러닝을 위한 프레임워크이다.
< 파이토치는 딥러닝을 위한 프레임워크이다. <EOS>

> pytorch is very clear to use.
= 파이토치는 사용하기 매우 직관적이다.
< 파이토치는 사용하기 매우 직관적이다. <EOS>

